<a href="https://colab.research.google.com/github/borhanur-rahman/Dengu_Biomarker_Discovery/blob/main/Dengu_Biomarker_Discovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Mount Drive & Installs
from google.colab import drive
drive.mount('/content/drive')

# Install required packages (gseapy works fine)
!pip install GEOparse gseapy scikit-learn xgboost keras tensorflow statsmodels rpy2 requests networkx py4cytoscape biopython

# No boruta_py → we use RF importance instead (paper's FCBF equivalent)
# For enrichment (like CKD notebook)
import gseapy as gp

# Imports (full set for pipeline)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
import GEOparse
from Bio import Entrez
import requests
import networkx as nx
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, classification_report, roc_curve
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense
Entrez.email = "borhanurrahman1@gmail.com"  # Required for NCBI (change to yours)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ------------------------------
# 2️⃣ Load Dengue Expression Matrix & Metadata from GEO
# ------------------------------

import GEOparse
import pandas as pd

# Target dataset from your slides: GSE51808 (PBMCs mild vs severe dengue)
gse_id = 'GSE51808'

print(f"Downloading and parsing {gse_id}...")
gse = GEOparse.get_GEO(geo=gse_id, destdir="./")

# Expression matrix: genes as rows, samples as columns
# 'VALUE' is usually the normalized/log-transformed intensity
expr = gse.pivot_samples('VALUE')

print("\nExpression matrix shape (genes × samples):", expr.shape)
print("\nFirst 5 genes and 5 samples:")
display(expr.iloc[:5, :5])

# Save expression to Drive (reuse later)
expr.to_csv('/content/drive/MyDrive/BioResearch/dengue_GSE51808_expression.csv')
print("Expression matrix saved to Drive.")

# ────────────────────────────────────────────────
# Inspect metadata columns (very important!)
# ────────────────────────────────────────────────
pheno = gse.phenotype_data.copy()

print("\nAll available metadata columns:")
print(pheno.columns.tolist())

print("\nFirst few rows of metadata:")
display(pheno.head())

# Look for columns that likely contain disease state
# Common candidates in GSE51808:
possible_columns = [col for col in pheno.columns if 'characteristic' in col.lower() or 'source' in col.lower() or 'disease' in col.lower() or 'status' in col.lower()]

print("\nColumns that probably contain severity info:")
print(possible_columns)

# ────────────────────────────────────────────────
# Create 'group' column – adjust based on print output above
# ────────────────────────────────────────────────
# Example: in GSE51808, severity is often in 'source_name_ch1' or 'characteristics_ch1'
# Run this AFTER you see the print output and choose the correct column

# Option 1: if severity is in 'source_name_ch1'
if 'source_name_ch1' in pheno.columns:
    pheno['group'] = pheno['source_name_ch1'].apply(
        lambda x: 'severe' if 'hemorrhagic' in str(x).lower() or 'severe' in str(x).lower() else 'non-severe'
    )

# Option 2: if severity is in 'characteristics_ch1' or similar (uncomment if needed)
# elif 'characteristics_ch1' in pheno.columns:
#     pheno['group'] = pheno['characteristics_ch1'].apply(
#         lambda x: 'severe' if 'hemorrhagic' in str(x).lower() or 'severe' in str(x).lower() else 'non-severe'
#     )

# Fallback: print unique values to help choose
print("\nUnique values in source_name_ch1 (if exists):")
if 'source_name_ch1' in pheno.columns:
    print(pheno['source_name_ch1'].unique())

# Show final group distribution
print("\nFinal group distribution (after assignment):")
print(pheno['group'].value_counts())

# Save metadata
pheno.to_csv('/content/drive/MyDrive/BioResearch/dengue_GSE51808_metadata.csv')
print("Metadata saved to Drive.")

26-Feb-2026 06:34:44 DEBUG utils - Directory ./ already exists. Skipping.
DEBUG:GEOparse:Directory ./ already exists. Skipping.
26-Feb-2026 06:34:44 INFO GEOparse - File already exist: using local version.
INFO:GEOparse:File already exist: using local version.
26-Feb-2026 06:34:44 INFO GEOparse - Parsing ./GSE51808_family.soft.gz: 
INFO:GEOparse:Parsing ./GSE51808_family.soft.gz: 
26-Feb-2026 06:34:44 DEBUG GEOparse - DATABASE: GeoMiame
DEBUG:GEOparse:DATABASE: GeoMiame
26-Feb-2026 06:34:44 DEBUG GEOparse - SERIES: GSE51808
DEBUG:GEOparse:SERIES: GSE51808
26-Feb-2026 06:34:44 DEBUG GEOparse - PLATFORM: GPL13158
DEBUG:GEOparse:PLATFORM: GPL13158


/usr/local/lib/python3.12/dist-packages/GEOparse/GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
26-Feb-2026 06:34:48 DEBUG GEOparse - SAMPLE: GSM1253028
DEBUG:GEOparse:SAMPLE: GSM1253028
26-Feb-2026 06:34:48 DEBUG GEOparse - SAMPLE: GSM1253029
DEBUG:GEOparse:SAMPLE: GSM1253029
26-Feb-2026 06:34:48 DEBUG GEOparse - SAMPLE: GSM1253030
DEBUG:GEOparse:SAMPLE: GSM1253030
26-Feb-2026 06:34:48 DEBUG GEOparse - SAMPLE: GSM1253031
DEBUG:GEOparse:SAMPLE: GSM1253031
26-Feb-2026 06:34:49 DEBUG GEOparse - SAMPLE: GSM1253032
DEBUG:GEOparse:SAMPLE: GSM1253032
26-Feb-2026 06:34:49 DEBUG GEOparse - SAMPLE: GSM1253033
DEBUG:GEOparse:SAMPLE: GSM1253033
26-Feb-2026 06:34:49 DEBUG GEOparse - SAMPLE: GSM1253034
DEBUG:GEOparse:SAMPLE: GSM1253034
26-Feb-2026 06:34:50 DEBUG GEOparse - SAMPLE: GSM1253035
DEBUG:GEOparse:SAMPLE: GSM1253035
26-Feb-2026 06:34:50 DEBUG GEOparse - SAMPLE: 


Expression matrix shape (genes × samples): (54715, 56)

First 5 genes and 5 samples:


name,GSM1253028,GSM1253029,GSM1253030,GSM1253031,GSM1253032
ID_REF,,,,,
1007_PM_s_at,5.97419,5.55657,6.51326,5.87408,5.36585
1053_PM_at,7.88225,7.75510,8.42061,7.78163,6.90308
117_PM_at,10.01840,8.97969,7.61888,10.45590,11.14070
121_PM_at,6.24138,6.55102,5.96311,5.99571,5.51645
1255_PM_g_at,2.92204,3.18026,2.67519,2.64095,2.76739


Expression matrix saved to Drive.

All available metadata columns:
['title', 'geo_accession', 'status', 'submission_date', 'last_update_date', 'type', 'channel_count', 'source_name_ch1', 'organism_ch1', 'taxid_ch1', 'characteristics_ch1.0.subject id', 'characteristics_ch1.1.infection', 'characteristics_ch1.2.status', 'characteristics_ch1.3.tissue', 'treatment_protocol_ch1', 'molecule_ch1', 'extract_protocol_ch1', 'label_ch1', 'label_protocol_ch1', 'hyb_protocol', 'scan_protocol', 'description', 'data_processing', 'platform_id', 'contact_name', 'contact_email', 'contact_phone', 'contact_laboratory', 'contact_institute', 'contact_address', 'contact_city', 'contact_state', 'contact_zip/postal_code', 'contact_country', 'supplementary_file', 'series_id', 'data_row_count']

First few rows of metadata:


,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,taxid_ch1,...,contact_laboratory,contact_institute,contact_address,contact_city,contact_state,contact_zip/postal_code,contact_country,supplementary_file,series_id,data_row_count
GSM1253028,Dengue Fever Patient 3,GSM1253028,Public on Jun 27 2014,Oct 28 2013,Jun 27 2014,RNA,1,Whole Blood of Dengue infected patient at acut...,Homo sapiens,9606,...,csbiology.com,Universidade de São Paulo,"AVENIDA PROFESSOR LINEU PRESTES, 580, Block 17",SÃO PAULO,SÃO PAULO,05508000,Brazil,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1253...,GSE51808,54715
GSM1253029,Dengue Fever Patient 4,GSM1253029,Public on Jun 27 2014,Oct 28 2013,Jun 27 2014,RNA,1,Whole Blood of Dengue infected patient at acut...,Homo sapiens,9606,...,csbiology.com,Universidade de São Paulo,"AVENIDA PROFESSOR LINEU PRESTES, 580, Block 17",SÃO PAULO,SÃO PAULO,05508000,Brazil,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1253...,GSE51808,54715
GSM1253030,Dengue Fever Patient 6,GSM1253030,Public on Jun 27 2014,Oct 28 2013,Jun 27 2014,RNA,1,Whole Blood of Dengue infected patient at acut...,Homo sapiens,9606,...,csbiology.com,Universidade de São Paulo,"AVENIDA PROFESSOR LINEU PRESTES, 580, Block 17",SÃO PAULO,SÃO PAULO,05508000,Brazil,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1253...,GSE51808,54715
GSM1253031,Dengue Fever Patient 7,GSM1253031,Public on Jun 27 2014,Oct 28 2013,Jun 27 2014,RNA,1,Whole Blood of Dengue infected patient at acut...,Homo sapiens,9606,...,csbiology.com,Universidade de São Paulo,"AVENIDA PROFESSOR LINEU PRESTES, 580, Block 17",SÃO PAULO,SÃO PAULO,05508000,Brazil,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1253...,GSE51808,54715
GSM1253032,Dengue Hemorrhagic Fever Patient 32,GSM1253032,Public on Jun 27 2014,Oct 28 2013,Jun 27 2014,RNA,1,Whole Blood of Dengue infected patient at acut...,Homo sapiens,9606,...,csbiology.com,Universidade de São Paulo,"AVENIDA PROFESSOR LINEU PRESTES, 580, Block 17",SÃO PAULO,SÃO PAULO,05508000,Brazil,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1253...,GSE51808,54715



Columns that probably contain severity info:
['status', 'source_name_ch1', 'characteristics_ch1.0.subject id', 'characteristics_ch1.1.infection', 'characteristics_ch1.2.status', 'characteristics_ch1.3.tissue']

Unique values in source_name_ch1 (if exists):
['Whole Blood of Dengue infected patient at acute infection time point'
 'Whole Blood of Dengue infected patient at convalescent time point'
 'Whole Blood of healthy control']

Final group distribution (after assignment):
group
non-severe    56
Name: count, dtype: int64
Metadata saved to Drive.


In [ ]:
pheno['group'] = pheno['title'].apply(
    lambda x: 'severe' if 'hemorrhagic fever' in str(x).lower() else 'non-severe'
)

In [ ]:
# ------------------------------
# 3️⃣ Preprocessing: Log Transform + Filter Low-Variance Genes
# ------------------------------

# Log2 transform (microarray data is often already normalized, but we add pseudo-count)
expr_log = np.log2(expr + 1)

# Filter low-variance genes (keep top 50% variance, like common practice)
var_threshold = np.percentile(expr_log.var(axis=0), 50)  # Median variance
expr_filtered = expr_log.loc[:, expr_log.var(axis=0) > var_threshold]

print("Filtered expression shape (genes × samples):", expr_filtered.shape)

# Save filtered data
expr_filtered.to_csv('/content/drive/MyDrive/BioResearch/dengue_GSE51808_filtered.csv')

Filtered expression shape (genes × samples): (54715, 28)


In [ ]:
# ------------------------------
# 3️⃣ Minimal Preprocessing (No Plots)
# ------------------------------

import numpy as np

# Safe log2 transform (GSE51808 is already normalized, but this is safe)
expr_log = np.log2(expr + 1)

# Basic variance filter (keep genes above median variance - fast)
var_threshold = np.percentile(expr_log.var(axis=1), 50)
expr_filtered = expr_log[expr_log.var(axis=1) > var_threshold]

print("Filtered shape (genes × samples):", expr_filtered.shape)
print(f"Kept {expr_filtered.shape[0]} genes after variance filter.")

# Align samples with groups (ensure index match)
common_samples = list(set(expr_filtered.columns) & set(pheno.index))
expr_filtered = expr_filtered[common_samples]
pheno = pheno.loc[common_samples]

print("Samples after alignment:", len(common_samples))

Filtered shape (genes × samples): (27357, 56)
Kept 27357 genes after variance filter.
Samples after alignment: 56


In [ ]:
# ------------------------------
# 3️⃣ Preprocessing + Variance Filter (Fixed & Safe – No Plots)
# ------------------------------

import numpy as np

# Safe log2 transform (handles any zeros/negatives)
expr_log = np.log2(expr + 1)
print("Log2 transform applied.")

# Calculate variance per gene (axis=0 = across samples/columns)
gene_variances = expr_log.var(axis=0)  # Series: index = genes (columns of expr_log)

# Median threshold
var_threshold = np.percentile(gene_variances, 50)
print(f"Median variance threshold: {var_threshold:.4f}")

# Boolean mask aligned with columns (genes)
keep_mask = gene_variances > var_threshold

# Filter columns using the mask (genes to keep)
# This avoids unalignable error by using .loc[:, keep_mask]
expr_filtered = expr_log.loc[:, keep_mask]

print("After variance filter → genes × samples:", expr_filtered.shape)
print(f"Kept {expr_filtered.shape[0]} genes out of {expr_log.shape[0]}.")

# Transpose for ML (samples as rows, genes as columns)
expr_ml = expr_filtered.T

# Align samples with pheno (final safety check)
common_samples = list(set(expr_ml.index) & set(pheno.index))
expr_ml = expr_ml.loc[common_samples]
pheno = pheno.loc[common_samples]

print("\nSamples after alignment:", len(common_samples))
print("Final ML-ready shape (samples × genes):", expr_ml.shape)

# Group check (should be ~46 non-severe, ~10 severe)
print("\nGroup distribution:")
print(pheno['group'].value_counts())

# Save ready data
expr_ml.to_csv('/content/drive/MyDrive/BioResearch/dengue_ml_ready.csv')
print("ML-ready data saved to Drive.")

Log2 transform applied.
Median variance threshold: 0.2480
After variance filter → genes × samples: (54715, 28)
Kept 54715 genes out of 54715.

Samples after alignment: 28
Final ML-ready shape (samples × genes): (28, 54715)

Group distribution:
group
non-severe    20
severe         8
Name: count, dtype: int64
ML-ready data saved to Drive.


In [ ]:
# ------------------------------
# 5️⃣ ML Models – Robust Version (Paper’s 6 Models + XGBoost – Stratified 5-Fold CV)
# ------------------------------

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
import numpy as np

# Use top genes by variance (adjust top_n for speed/accuracy)
top_n = 2000  # 2000 is fast; try 5000 or 10000 later
gene_vars = expr_ml.var(axis=0)
top_genes = gene_vars.nlargest(top_n).index.tolist()

X = expr_ml[top_genes].values  # numpy for consistency
y = pheno['group'].map({'non-severe': 0, 'severe': 1}).values

print(f"Training with {X.shape[1]} genes | Samples: {X.shape[0]} | Classes: {np.bincount(y)}")

# Stratified 5-fold CV (better than single split for small/imbalanced data)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

models = {
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Naive Bayes': GaussianNB(),
    'AdaBoost': AdaBoostClassifier(n_estimators=50, random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')
}

results = {name: {'acc': [], 'prec': [], 'rec': [], 'auc': []} for name in models}

for train_idx, test_idx in skf.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Safe probability extraction (some models return shape (n,1) or (n,))
        if hasattr(model, 'predict_proba'):
            y_prob = model.predict_proba(X_test)
            if y_prob.ndim == 2 and y_prob.shape[1] == 2:
                y_prob = y_prob[:, 1]  # probability of class 1 (severe)
            else:
                y_prob = y_prob.flatten()  # fallback
        else:
            y_prob = y_pred.astype(float)  # fallback to binary

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        auc = roc_auc_score(y_test, y_prob) if len(np.unique(y_test)) > 1 else np.nan

        results[name]['acc'].append(acc)
        results[name]['prec'].append(prec)
        results[name]['rec'].append(rec)
        results[name]['auc'].append(auc)

# Average performance
results_df = pd.DataFrame({
    name: {
        'Accuracy': np.mean(res['acc']),
        'Precision': np.mean(res['prec']),
        'Recall (Sensitivity)': np.mean(res['rec']),
        'AUC': np.nanmean(res['auc'])  # nanmean ignores N/A
    } for name, res in results.items()
}).T

print("\nML Model Performance (5-Fold Stratified CV – Average):")
print(results_df.round(4))

# Best model
best_auc = results_df['AUC'].idxmax() if results_df['AUC'].notna().any() else 'N/A'
best_acc = results_df['Accuracy'].idxmax()
print(f"\nBest model by AUC: {best_auc}")
print(f"Best model by Accuracy: {best_acc}")

Training with 2000 genes | Samples: 28 | Classes: [20  8]

ML Model Performance (5-Fold Stratified CV – Average):
               Accuracy  Precision  Recall (Sensitivity)     AUC
KNN              0.8600     0.7667                   0.9  0.8875
Naive Bayes      0.8933     0.8333                   0.9  0.9000
AdaBoost         0.8267     0.7667                   0.8  0.8250
SVM              0.7933     0.6000                   0.6  0.9250
Random Forest    0.9000     0.7333                   0.8  0.9500
XGBoost          0.8600     0.8333                   0.8  0.9000

Best model by AUC: Random Forest
Best model by Accuracy: Random Forest


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from imblearn.over_sampling import SMOTE  # For imbalance
import shap  # For biomarker prediction (explainability)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

# Load your dengue data (from previous saved file or GEO)
# If you have 'dengue_ml_ready.csv' from earlier
expr_ml = pd.read_csv('/content/drive/MyDrive/BioResearch/dengue_ml_ready.csv', index_col=0)  # samples × genes

# If not, load from GEO (uncomment if needed)
# import GEOparse
# gse = GEOparse.get_GEO(geo='GSE51808', destdir="./")
# expr = gse.pivot_samples('VALUE')
# expr_log = np.log2(expr + 1)
# expr_ml = expr_log.T  # samples × genes

# Pheno (groups) – assume you have it; if not, recreate
pheno = pd.read_csv('/content/drive/MyDrive/BioResearch/dengue_GSE51808_metadata_fixed.csv', index_col=0)
y = pheno['group'].map({'non-severe': 0, 'severe': 1})

print("Data shape (samples × genes):", expr_ml.shape)
print("Labels distribution:", y.value_counts().to_dict())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Data shape (samples × genes): (28, 54715)
Labels distribution: {0: 46, 1: 10}


In [ ]:
# ------------------------------
# 3️⃣ Preprocessing + Variance Filter (Fixed & Robust – No Plots)
# ------------------------------

import numpy as np

# Safe log2 transform
expr_log = np.log2(expr + 1)
print("Log2 transform applied.")

# Calculate variance per gene (axis=0 = across samples/columns)
gene_variances = expr_log.var(axis=0)  # Series with gene names as index

# Median threshold
var_threshold = np.percentile(gene_variances, 50)
print(f"Median variance threshold: {var_threshold:.4f}")

# Create boolean mask using the exact column index (genes)
# This ensures perfect alignment
keep_mask = pd.Series(False, index=expr_log.columns)
keep_mask[gene_variances.index] = gene_variances > var_threshold

# Filter using .loc on columns
expr_filtered = expr_log.loc[:, keep_mask]

print("After variance filter → genes × samples:", expr_filtered.shape)
print(f"Kept {expr_filtered.shape[0]} genes out of {expr_log.shape[0]}.")

# Transpose for ML (samples as rows, genes as columns)
expr_ml = expr_filtered.T

# Align samples with pheno (final safety check)
common_samples = list(set(expr_ml.index) & set(pheno.index))
expr_ml = expr_ml.loc[common_samples]
pheno = pheno.loc[common_samples]

print("\nSamples after alignment:", len(common_samples))
print("Final ML-ready shape (samples × genes):", expr_ml.shape)

# Group check (should show non-severe ~20, severe ~8)
print("\nGroup distribution:")
print(pheno['group'].value_counts())

# Save ready data
expr_ml.to_csv('/content/drive/MyDrive/BioResearch/dengue_ml_ready.csv')
print("ML-ready data saved to Drive.")

Log2 transform applied.
Median variance threshold: 0.2480
After variance filter → genes × samples: (54715, 28)
Kept 54715 genes out of 54715.

Samples after alignment: 28
Final ML-ready shape (samples × genes): (28, 54715)

Group distribution:
group
non-severe    20
severe         8
Name: count, dtype: int64
ML-ready data saved to Drive.


In [ ]:
# ------------------------------
# Cell: Build & Train Custom DNN Model for Dengue Severity
# ------------------------------

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE  # Handle imbalance
import shap  # For biomarker discovery
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Use all genes or subsample for speed (change top_n if needed)
top_n = 5000  # 5000 is good balance; increase to 10000+ for more power
gene_vars = expr_ml.var(axis=0)
top_genes = gene_vars.nlargest(top_n).index.tolist()

X = expr_ml[top_genes].values  # numpy array
y = pheno['group'].map({'non-severe': 0, 'severe': 1}).values

print(f"Using {X.shape[1]} genes | Samples: {X.shape[0]} | Classes: {np.bincount(y)}")

# Normalize features (important for DNN)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Handle imbalance with SMOTE (oversample severe class)
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)
print("After SMOTE:", X_res.shape, np.bincount(y_res))

# Split (80/20 stratified)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, stratify=y_res, random_state=42)

# Custom DNN architecture (3 hidden layers + dropout for regularization)
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))  # Input layer
model.add(Dropout(0.3))  # Prevent overfitting
model.add(Dense(128, activation='relu'))  # Hidden layer 1
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))  # Hidden layer 2
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Output (binary: 0/1)

# Compile
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to avoid overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train (10–50 epochs, class weights if needed)
class_weights = {0: 1.0, 1: len(y_res[y_res==0]) / len(y_res[y_res==1])} if len(np.unique(y_res)) > 1 else None
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_split=0.2,
    class_weight=class_weights,
    callbacks=[early_stop],
    verbose=1
)

# Evaluate on test set
loss, acc = model.evaluate(X_test, y_test, verbose=0)
y_prob = model.predict(X_test).flatten()
y_pred = (y_prob > 0.5).astype(int)

prec = precision_score(y_test, y_pred, zero_division=0)
rec = recall_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_prob)

print("\nCustom DNN Performance on Test Set:")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall (Sensitivity): {rec:.4f}")
print(f"AUC: {auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Non-Severe', 'Severe']))

Using 5000 genes | Samples: 28 | Classes: [20  8]
After SMOTE: (40, 5000) [20 20]
Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step - accuracy: 0.5933 - loss: 0.7957 - val_accuracy: 0.8571 - val_loss: 0.6334
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.8517 - loss: 0.5891 - val_accuracy: 0.8571 - val_loss: 0.3548
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.8992 - loss: 0.4333 - val_accuracy: 0.8571 - val_loss: 0.1349
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.9525 - loss: 0.6743 - val_accuracy: 1.0000 - val_loss: 0.0340
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.8517 - loss: 0.6314 - val_accuracy: 1.0000 - val_loss: 0.0201
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.8571 - val_loss: 0.1310
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 1.0000 - loss: 0.0110 - val_accuracy: 0.8571 - val_loss: 0.3729
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.9525 - loss: 0.1312 - val_accuracy: 0.8571 - val_loss: 0.5163
Epoch 9/50

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step

Custom DNN Performance on Test Set:
Accuracy: 0.8750
Precision: 1.0000
Recall (Sensitivity): 0.7500
AUC: 0.8750

Classification Report:
              precision    recall  f1-score   support

  Non-Severe       0.80      1.00      0.89         4
      Severe       1.00      0.75      0.86         4

    accuracy                           0.88         8
   macro avg       0.90      0.88      0.87         8
weighted avg       0.90      0.88      0.87         8



In [ ]:
# ------------------------------
# 6️⃣ Predict Novel Biomarkers with SHAP (on Best Model: Random Forest)
# ------------------------------

import shap
import pandas as pd
import numpy as np

# Use the trained Random Forest model
best_model = models['Random Forest']

print("Number of genes used:", len(top_genes))

# Convert to DataFrame (SHAP prefers feature names)
X_test_used = pd.DataFrame(X_test, columns=top_genes)

print("Computing SHAP values on test set...")

# TreeExplainer (best for Random Forest)
explainer = shap.TreeExplainer(best_model)

shap_values = explainer.shap_values(X_test_used)

# ------------------------------
# 🔥 Handle all possible SHAP formats safely
# ------------------------------

# Case 1️⃣: Old versions → list of arrays [class0, class1]
if isinstance(shap_values, list):
    shap_positive = shap_values[1]   # class 1 (severe dengue)

# Case 2️⃣: New versions → 3D array (n_samples, n_features, n_classes)
elif isinstance(shap_values, np.ndarray) and shap_values.ndim == 3:
    shap_positive = shap_values[:, :, 1]  # select class 1

# Case 3️⃣: Already 2D (n_samples, n_features)
else:
    shap_positive = shap_values

print("SHAP values shape (positive class):", shap_positive.shape)

# ------------------------------
# Compute Mean |SHAP| per gene
# ------------------------------

shap_abs_mean = np.abs(shap_positive).mean(axis=0)

print("Length of top_genes:", len(top_genes))
print("Length of SHAP mean:", len(shap_abs_mean))

# ------------------------------
# Build Importance DataFrame
# ------------------------------

shap_importance = pd.DataFrame({
    'gene': top_genes,
    'shap_mean_abs': shap_abs_mean
})

shap_importance = shap_importance.sort_values(
    'shap_mean_abs',
    ascending=False
).reset_index(drop=True)

print("\nTop 20 Novel Biomarker Candidates (by SHAP importance for severe dengue prediction):")
display(shap_importance.head(20))

# ------------------------------
# Save Top 100
# ------------------------------

shap_importance.head(100).to_csv(
    '/content/drive/MyDrive/BioResearch/dengue_novel_biomarkers_SHAP_RF.csv',
    index=False
)

print("Top 100 SHAP-based biomarkers saved to Drive.")

Number of genes used: 5000
Computing SHAP values on test set...
SHAP values shape (positive class): (8, 5000)
Length of top_genes: 5000
Length of SHAP mean: 5000

Top 20 Novel Biomarker Candidates (by SHAP importance for severe dengue prediction):


,gene,shap_mean_abs
0,233011_PM_at,0.020000
1,205114_PM_s_at,0.009557
2,210549_PM_s_at,0.008880
3,240182_PM_at,0.008650
4,217518_PM_at,0.008385
5,227671_PM_at,0.008261
6,232397_PM_at,0.008261
7,202587_PM_s_at,0.008261
8,212845_PM_at,0.007826
9,226817_PM_at,0.007826


Top 100 SHAP-based biomarkers saved to Drive.


In [ ]:
# ------------------------------
# ML Models – Direct Jump (Paper’s 6 Models + XGBoost – 5-Fold CV)
# ------------------------------

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

# Select top genes by variance (fast & sufficient for n=28)
top_n = 2000  # Change to 5000 or expr_ml.shape[1] for more power
gene_vars = expr_ml.var(axis=0)
top_genes = gene_vars.nlargest(top_n).index.tolist()

X = expr_ml[top_genes].values  # numpy array for speed
y = pheno['group'].map({'non-severe': 0, 'severe': 1}).values

print(f"Training with {X.shape[1]} genes | Samples: {X.shape[0]} | Classes: {np.bincount(y)}")

# Stratified 5-fold CV (more reliable than single split)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

models = {
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Naive Bayes': GaussianNB(),
    'AdaBoost': AdaBoostClassifier(n_estimators=50, random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')
}

results = {name: {'acc': [], 'prec': [], 'rec': [], 'auc': []} for name in models}

for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), 1):
    print(f"\nFold {fold}/5")
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Safe probability extraction
        if hasattr(model, 'predict_proba'):
            y_prob = model.predict_proba(X_test)
            if y_prob.ndim == 2 and y_prob.shape[1] == 2:
                y_prob = y_prob[:, 1]  # prob of class 1 (severe)
            else:
                y_prob = y_prob.flatten()  # fallback
        else:
            y_prob = y_pred.astype(float)

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        auc = roc_auc_score(y_test, y_prob) if len(np.unique(y_test)) > 1 else np.nan

        results[name]['acc'].append(acc)
        results[name]['prec'].append(prec)
        results[name]['rec'].append(rec)
        results[name]['auc'].append(auc)

# Average performance across folds
results_df = pd.DataFrame({
    name: {
        'Accuracy': np.mean(res['acc']),
        'Precision': np.mean(res['prec']),
        'Recall (Sensitivity)': np.mean(res['rec']),
        'AUC': np.nanmean(res['auc'])
    } for name, res in results.items()
}).T

print("\nML Model Performance (5-Fold Stratified CV – Average):")
print(results_df.round(4))

# Best model
best_auc = results_df['AUC'].idxmax() if results_df['AUC'].notna().any() else 'N/A'
best_acc = results_df['Accuracy'].idxmax()
print(f"\nBest model by AUC: {best_auc}")
print(f"Best model by Accuracy: {best_acc}")

Training with 2000 genes | Samples: 28 | Classes: [20  8]

Fold 1/5

Fold 2/5

Fold 3/5

Fold 4/5

Fold 5/5

ML Model Performance (5-Fold Stratified CV – Average):
               Accuracy  Precision  Recall (Sensitivity)     AUC
KNN              0.8600     0.7667                   0.9  0.8875
Naive Bayes      0.8933     0.8333                   0.9  0.9000
AdaBoost         0.8267     0.7667                   0.8  0.8250
SVM              0.7933     0.6000                   0.6  0.9250
Random Forest    0.9000     0.7333                   0.8  0.9500
XGBoost          0.8600     0.8333                   0.8  0.9000

Best model by AUC: Random Forest
Best model by Accuracy: Random Forest


In [ ]:
# ------------------------------
# 6️⃣ Predict Novel Biomarkers with SHAP (on Best Model: Random Forest)
# ------------------------------

import shap
import pandas as pd
import numpy as np

# Use the trained Random Forest model (best by AUC from your output)
best_model = models['Random Forest']

# Features used (top_genes from previous cell)
print("Number of genes used:", len(top_genes))

# Prepare X_test_used as DataFrame with gene names (SHAP likes feature names)
X_test_used = pd.DataFrame(X_test, columns=top_genes)

print("Computing SHAP values on test set... (should be fast for RF, 30s–2min)")

# TreeExplainer for Random Forest (fast & accurate)
explainer = shap.TreeExplainer(best_model)

# Compute SHAP values (returns list for multi-class, but binary often [neg, pos])
shap_values = explainer.shap_values(X_test_used)

# Handle binary case safely
if isinstance(shap_values, list):
    # Take SHAP for positive class (severe = 1)
    shap_positive = shap_values[1]  # shape: (n_test, n_features)
else:
    shap_positive = shap_values  # if already array for positive class

print("SHAP values shape:", shap_positive.shape)

# Average absolute SHAP per gene (importance score)
shap_abs_mean = np.abs(shap_positive).mean(axis=0)  # length = n_features

# Confirm lengths match
print("Length of top_genes:", len(top_genes))
print("Length of SHAP mean:", len(shap_abs_mean))

# Build DataFrame (lengths must match now)
shap_importance = pd.DataFrame({
    'gene': top_genes,
    'shap_mean_abs': shap_abs_mean
}).sort_values('shap_mean_abs', ascending=False).reset_index(drop=True)

print("\nTop 20 Novel Biomarker Candidates (by SHAP importance for severe dengue prediction):")
display(shap_importance.head(20))

# Save top 100 candidates
shap_importance.head(100).to_csv(
    '/content/drive/MyDrive/BioResearch/dengue_novel_biomarkers_SHAP_RF.csv',
    index=False
)
print("Top 100 SHAP-based biomarkers saved to Drive.")

Number of genes used: 5000
Computing SHAP values on test set... (should be fast for RF, 30s–2min)
SHAP values shape: (8, 5000, 2)
Length of top_genes: 5000
Length of SHAP mean: 5000


ValueError: Per-column arrays must each be 1-dimensional